Cuarto ejercicio de Spark Structured Streaming. WordCount de los datos consumidos desde un topic de kafka.
NOTA: Es estrictamente necesario editar la configuración del fichero kernel.json para que cargue como dependencia la biblioteca org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0

In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import explode 
from pyspark.sql.functions import split

In [ ]:
spark = SparkSession\
        .builder\
        .appName("StructuredKafkaWordCount")\
        .getOrCreate()

In [ ]:
# Create DataSet representing the stream of input lines from kafka
# Es necesario de antemano haber creado el topic llamado wordcount_topic
lines = spark\
        .readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", "localhost:9092")\
        .option("subscribe", "wordcount_topic")\
        .load()\
        .selectExpr("CAST(value AS STRING)")

In [ ]:
# Split the lines into words 
words = lines.select( 
    # explode turns each item in an array into a separate row 
    explode( 
        split(lines.value, ' ') 
        ).alias('word') 
    )

In [ ]:
# Generate running word count 
wordCounts = words.groupBy('word').count() 

In [ ]:
# Start running the query that prints the running counts to the console
# Una vez iniciado el procesamiento del flujo empezar a insertar elementos al topic de kafka
# Ver resultados en la consola donde se ejecuto el jupyter notebook
query = wordCounts\
    .writeStream\
    .outputMode('complete')\
    .format('console')\
    .start() 

query.awaitTermination() 